Notebook to backtest with backtrader the ML scores built for smart DCA on CW8

In [98]:
from importlib import reload
import pandas as pd
import backtrader as bt
import bt_ml_strategies as strat
reload(strat)

PATH_DATA_BT = "../../Data/backtest/"



In [123]:
file_name="DCA_CLOSE_1D_21D_V1_lab_perf_21d_predict_lstm_v3_e_BT.zip"
df = pd.read_csv(PATH_DATA_BT+file_name, sep=",",
                            index_col="OPEN_DATETIME", parse_dates=True, keep_default_na=True, low_memory=False)

In [ ]:
prices=df[['open','high','low','close','volume','Predict','SL','TP']]
prices[['SL','TP']]=0
prices=prices['2010-04-01 00:00:00':] #start date of the backtest
prices.dropna(inplace=True)
prices.sort_index(inplace=True)
prices.info()

In [125]:
data = strat.SignalData(dataname=prices,timeframe=bt.TimeFrame.Days)#, compression=30)

In [ ]:
# Tune dca_params according your strategie
# dca_params = {'score_buy': 0.00, 'score_sell': -10.005, 'size_buy': 2500, 'size_sell': 2500} # BACKTEST FOR TRUE DCA CW8 REGRESSION SCORE

# dca_params = {'score_buy': 2.00, 'score_sell': 0.00, 'size_buy': 2500, 'size_sell': 2500} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 1
# dca_params = {'score_buy': 2.00, 'score_sell': 0.50, 'size_buy': 2500, 'size_sell': 2500, 'reinvest_split':1} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 2
# dca_params = {'score_buy': 3.00, 'score_sell': 0.00, 'size_buy': 2500, 'size_sell': 2500} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 3
dca_params = {'score_buy': 1.00, 'score_sell': 0.00, 'size_buy': 2500, 'size_sell': 2500} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 4
# dca_params = {'score_buy': 1.00, 'score_sell': 0.50, 'size_buy': 2500, 'size_sell': 2500, 'reinvest_split':1} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 5
# dca_params = {'score_buy': 1.00, 'score_sell': 0.50, 'size_buy': 2500, 'size_sell': 2500, 'reinvest_split':2} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 6
# dca_params = {'score_buy': 2.00, 'score_sell': 0.50, 'size_buy': 2500, 'size_sell': 2500, 'reinvest_split':2} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 7
# dca_params = {'score_buy': 2.00, 'score_sell': 1.50, 'size_buy': 2500, 'size_sell': 2500, 'reinvest_split':1} # BACKTEST FOR CW8 LSTM CLASSIFICATION SCORE SCENARIO 8
cerebro = bt.Cerebro(cheat_on_open=True)
cerebro.addstrategy(strat.BtMlDcaStrategy,**dca_params)
cerebro.adddata(data)
cerebro.broker.setcash(1000000.0)
cerebro.broker.setcommission(commission=0.05) #0.5% commission !
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")

print(f"Starting portfolio:{cerebro.broker.getvalue()}")
backtest_result=cerebro.run()
print(f"Ending portfolio:{cerebro.broker.getvalue()}")